<a href="https://colab.research.google.com/github/dev-chaitanya-dewangan/CHATGPT_TRANSFORM_FROM_SCRATCH/blob/main/LLM_FROM_SCRATCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA PREP

In [2]:
import requests
import re
url = "https://www.gutenberg.org/cache/epub/76137/pg76137.txt"
response = requests.get(url)

# Get the text content
full_text = response.text
first_index = full_text.find("PREFACE")
second_index = full_text.find("PREFACE", first_index + 1)
# Preview first 500 characters
book= full_text[second_index:]

In [3]:
preproccesed_cleaned_text = re.split(r'([,.:;?_!"()\']|--|\s)',book)
preporcessed_result=[item.strip() for item in preproccesed_cleaned_text if item.strip()]

# **EXTENED WITH UNK AND ENDOF TEXT**

In [4]:
preporcessed_sorted_result=sorted(set(preporcessed_result))
preporcessed_sorted_result.extend(["<|endoftext|>","<|unk|>"])

In [5]:

vocab={token:id for id,token in enumerate(preporcessed_sorted_result)}


# TOKENIZER


In [6]:
class Tokenizer:
  def __init__(self,vocab):
    self.encoded_vocab=vocab
    self.decoded_vocab={id:word for word,id in vocab.items()}

  def encode(self,text):
    text_cleaning=re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preporccesed_text=[item.strip() for item in text_cleaning if item.strip()]
    tokenized_text = [self.encoded_vocab[i] for i in preporccesed_text]
    return tokenized_text
  def decode(self,tokens):
    text=" ".join([self.decoded_vocab[i] for i in tokens])
    text=re.sub('\s+([,.?!"()\'])',r'\1',text)
    return text
tokenizer=Tokenizer(vocab)
text="A"
tokenizer.decode([0]),tokenizer.encode(text)

('!', [471])

# NEW TOKENIZER EXTENDED WITH THE UNKOWN TEXT AND END OF TEXT\

In [7]:

class TokenizerV2:
  def __init__(self,vocab):
    self.encoded_vocab=vocab
    self.decoded_vocab={id:word for word,id in vocab.items()}

  def encode(self,text):
    text_cleaning=re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preporccesed_text=[item.strip() for item in text_cleaning if item.strip()]
    # tokenized_text = [self.encoded_vocab[i] for i in preporccesed_text] CHANGED HERE
    tokenized_text = [i if i in self.encoded_vocab else "<|unk|>" for i in preporccesed_text]
    return tokenized_text
  def decode(self,tokens):
    text=" ".join([self.decoded_vocab[i] for i in tokens])
    text=re.sub('\s+([,.?!"()\'])',r'\1',text)
    return text
tokenizer=TokenizerV2(vocab)
text="aaa"
tokenizer.encode(text)

['<|unk|>']

In [8]:
import tiktoken
import torch
from torch.utils.data import Dataset,DataLoader
class GPTEncoder(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input=[]
    self.taget=[]
    tokenized_text=tokenizer.encode(txt,allowed_special="<|endoftext|>")
    for i in range(0,len(tokenized_text)-max_length,stride):
      input=tokenized_text[i:i+max_length]
      target=tokenized_text[i+1:1+i+max_length]
      self.input.append(torch.tensor(input))
      self.target.append(torch.tensor(target))
  def __len__(self):
    return len(self.input)
  def __getitem__(self,pos):
    return self.input[pos],self.target[pos]

def create_dataloader(txt,batch_size=4,max_length=256
                      ,stride=128,shuffle=True
                      ,drop_last=True,num_workers=0):
  tokenizer=tiktoken.get_encoding("gpt2")
  dataset = GPTEncoder(txt,tokenizer,max_length,stride)
  dataloader=DataLoader(
      batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers
  )